In [1]:
import folium
import pandas as pd
import json
from folium import plugins
import geocoder
from geopy.distance import great_circle

In [ ]:

with open('전국_정신건강관련기관_위치정보.json', 'r', encoding='utf-8') as f:
    loc_data = json.load(f)

loc_arr = loc_data['data']
loc_arr

df = pd.DataFrame(loc_arr)

df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

df.loc[df['기관구분'] == '정신병원', '기관구분'] = '병원'

df.index = range(len(df))
df


class MappingByCoord:

    def __init__(self, df, lat, lng, dist=5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist

    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
            ]
        result.index = range(len(result))

        return result

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

        result.index = range(len(result))

        return result

    def MappingInRectangler(self, df):

        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)

        for idx, row in df.iterrows():
            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m

    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], width='70%', height='100%', zoom_start=13)
        mcg = folium.plugins.MarkerCluster(control=False)
        m.add_child(mcg)

        sangdam = folium.plugins.FeatureGroupSubGroup(mcg, "상담소")
        center_ = folium.plugins.FeatureGroupSubGroup(mcg, "센터")
        ins = folium.plugins.FeatureGroupSubGroup(mcg, "시설")
        hos = folium.plugins.FeatureGroupSubGroup(mcg, "병원")
        bogun = folium.plugins.FeatureGroupSubGroup(mcg, "보건소")

        m.add_child(sangdam)
        m.add_child(center_)
        m.add_child(ins)
        m.add_child(hos)
        m.add_child(bogun)

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            div = ['상담소', '센터', '시설', '병원', '보건소']

            if row['기관구분'] == div[0]:
                sangdam.add_child(
                    folium.Marker([lat_now, lng_now], icon=folium.Icon(color='pink'), radius=15, tooltip=row['기관명']))

            elif row['기관구분'] == div[1]:
                center_.add_child(
                    folium.Marker([lat_now, lng_now], icon=folium.Icon(color='green'), radius=15, tooltip=row['기관명']))

            elif row['기관구분'] == div[2]:
                ins.add_child(
                    folium.Marker([lat_now, lng_now], icon=folium.Icon(color='blue'), radius=15, tooltip=row['기관명']))

            elif row['기관구분'] == div[3]:
                hos.add_child(
                    folium.Marker([lat_now, lng_now], icon=folium.Icon(color='purple'), radius=15, tooltip=row['기관명']))

            else:
                bogun.add_child(
                    folium.Marker([lat_now, lng_now], icon=folium.Icon(color='orange'), radius=15, tooltip=row['기관명']))

        folium.Circle(radius=self.dist * 1000,
                      location=[self.lat, self.lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m

def MakeMap(request):
    user_loc = geocoder.ip('me')
    mylocation = user_loc.latlng

    lat = mylocation[0]
    lng = mylocation[1]
    dist = 5

    df = pd.DataFrame()
    
    

    mbc = MappingByCoord(df, lat, lng, dist)

    result_radius = mbc.setCircle()

    mymap = mbc.MappingInCircle(result_radius)
    folium.Marker(location=mylocation, popup='현재 나의 위치', icon=folium.Icon(color='red', icon='star')).add_to(mymap)
    plugins.LocateControl().add_to(mymap)
    plugins.Geocoder(position='bottomright').add_to(mymap)
    folium.LayerControl(collapsed=False, position='bottomright').add_to(mymap)

    maps = mymap._repr_html_()
    return render(request, '../templates/map_show.html', {'mymap': maps})


In [3]:
loc_all = class.objects.all().values()
df_test = pd.DataFrame(loc_all)

df_test.columns

""
